In [1]:
import pandas as pd

In [2]:
df=pd.read_csv("../../datasets/nextword.txt",delimiter='\t')

In [3]:
df.columns=['nextword']

In [4]:
df

,nextword
0,This eBook is for the use of anyone anywhere a...
1,almost no restrictions whatsoever. You may co...
2,re-use it under the terms of the Project Guten...
3,with this eBook or online at www.gutenberg.net
4,Title: The Adventures of Sherlock Holmes
...,...
9627,facility: www.gutenberg.org
9628,This Web site includes information about Proje...
9629,including how to make donations to the Project...
9630,"Archive Foundation, how to help produce our ne..."


In [5]:
import re
def clean(r0):
    return re.sub('<.*?>',' ',r0)

In [6]:
df['nextword'] = df['nextword'].apply(clean)

In [7]:
from tensorflow.keras.preprocessing.text import Tokenizer

In [8]:
tokenizer = Tokenizer(num_words=5000)
tokenizer.fit_on_texts(df['nextword'])

In [9]:
total_words=len(tokenizer.index_word)+1

In [10]:
total_words

8930

In [11]:
input_sequences = []
for line in df['nextword']:
    tokens = tokenizer.texts_to_sequences([line])[0]
    for i in range(1, len(tokens)):
        n_gram_sequence = tokens[:i+1]
        input_sequences.append(n_gram_sequence)

In [12]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
max_sequence_length = max(len(seq) for seq in input_sequences)
input_sequences = pad_sequences(input_sequences, maxlen=max_sequence_length, padding='pre')


In [13]:
X = input_sequences[:, :-1]
y = input_sequences[:, -1]

In [14]:
from tensorflow.keras.utils import to_categorical
y = to_categorical(y, num_classes=total_words)

In [15]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense

model = Sequential()
model.add(Embedding(total_words, 128, input_length=max_sequence_length - 1))
model.add(LSTM(50, return_sequences=True))
model.add(LSTM(50, return_sequences=True))
model.add(LSTM(50, return_sequences=True))
model.add(LSTM(50, return_sequences=False))
model.add(Dense(total_words, activation='softmax'))

model.compile(loss='categorical_crossentropy', metrics=['accuracy'])


In [16]:
model.fit(X, y, epochs=100,batch_size=64)


Epoch 1/100
1527/1527 [==============================] - 104s 62ms/step - loss: 6.5430 - accuracy: 0.0543
Epoch 2/100
1527/1527 [==============================] - 96s 63ms/step - loss: 6.4321 - accuracy: 0.0544
Epoch 3/100
1527/1527 [==============================] - 97s 64ms/step - loss: 6.4319 - accuracy: 0.0544
Epoch 4/100
1527/1527 [==============================] - 96s 63ms/step - loss: 6.4315 - accuracy: 0.0544
Epoch 5/100
1527/1527 [==============================] - 95s 62ms/step - loss: 6.4305 - accuracy: 0.0544
Epoch 6/100
1527/1527 [==============================] - 98s 64ms/step - loss: 6.4313 - accuracy: 0.0544
Epoch 7/100
1527/1527 [==============================] - 97s 63ms/step - loss: 6.4318 - accuracy: 0.0544
Epoch 8/100
1527/1527 [==============================] - 95s 62ms/step - loss: 6.4313 - accuracy: 0.0544
Epoch 9/100
1527/1527 [==============================] - 95s 62ms/step - loss: 6.4294 - accuracy: 0.0544
Epoch 10/100
1527/1527 [==============================

1527/1527 [==============================] - 95s 63ms/step - loss: 4.4141 - accuracy: 0.2936
Epoch 79/100
1527/1527 [==============================] - 98s 64ms/step - loss: 4.4008 - accuracy: 0.2943
Epoch 80/100
1527/1527 [==============================] - 100s 65ms/step - loss: 4.3901 - accuracy: 0.2968
Epoch 81/100
1527/1527 [==============================] - 98s 64ms/step - loss: 4.3751 - accuracy: 0.2993
Epoch 82/100
1527/1527 [==============================] - 98s 64ms/step - loss: 4.3635 - accuracy: 0.3002
Epoch 83/100
1527/1527 [==============================] - 96s 63ms/step - loss: 4.3512 - accuracy: 0.3031
Epoch 84/100
1527/1527 [==============================] - 96s 63ms/step - loss: 4.3394 - accuracy: 0.3046
Epoch 85/100
1527/1527 [==============================] - 96s 63ms/step - loss: 4.3264 - accuracy: 0.3063
Epoch 86/100
1527/1527 [==============================] - 97s 64ms/step - loss: 4.3149 - accuracy: 0.3080
Epoch 87/100
1527/1527 [==============================] - 

In [17]:
model.save('nextword.h5')

C:\ProgramData\anaconda3\envs\dsml_25_tf\lib\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [18]:
from tensorflow.keras.models import load_model

In [19]:
model = load_model('nextword.h5')

In [20]:
test_data = ['it includes']

In [21]:
test_tokenized = tokenizer.texts_to_sequences(test_data)

In [24]:
test_padded_reviews = pad_sequences(test_tokenized,19)

In [25]:
model.predict(test_padded_reviews)

1/1 [==============================] - 6s 6s/step


array([[7.5060188e-06, 2.0792428e-03, 4.9238544e-02, ..., 7.7261084e-06,
        7.4351165e-06, 7.4710533e-06]], dtype=float32)